In [2]:
# importamos las librerías con las que vamos a trabajar

# Trabajar con bases de datos y python
# -----------------------------------------------------------------------
import psycopg2
from psycopg2 import OperationalError, errorcodes, errors


# Trabajar con DataFrames
# -----------------------------------------------------------------------
import pandas as pd



In [3]:
pd.options.display.max_columns = None

In [4]:
df_4sq = pd.read_csv("../../Datos/api_foursquare.csv", index_col= 0)
df_aemet = pd.read_csv("../../Datos/df_aemet_final.csv")
df_municipios = pd.read_csv("../../Datos/municipios.csv",index_col = 0)

Plantearme 5 preguntas con estos datos:



-Eres una empresa de grabación que quiere grabar una película en un parque en los alreedores de Madrid, cuál es el sitio con el mejor tiempo en gral?

-En qué municipio hace más sol para grabar

-Dónde puedo encontrar un castillo para grabar un episodio medieval? Necesito que haya un estudio de películas cerca.

-Dónde hay más probabilidad de que nieve?

-Hay que evitar los lugares muy húmedos, pues afectan a las cámaras


In [5]:
#Análisis exploratorio
##df_4sq tiene información de lugares de interés en los diferentes municipios, sus horarios, nombre, dirección y distancia desde el centro. tb coords. 
##df_aemet tiene información del tiempo en días determinados en determinados lugares
##df_municipios tiene información de los municipios y su localización con coords.



In [6]:
df_aemet.head()

,fecha,cielo,temp._(°c),sen._térmica_(°c),racha_máx._(km/h),precipitación_(mm),nieve_(mm),humedad_relativa_(%),prob._precip.__(%),prob._de_nieve_(%),prob._de_tormenta_(%),avisos,dirección_viento,velocidad_del_viento,localizacion_id
0,2024-08-29 12:00:00,Nubes altas,21,21,25.0,0.0,0,69,95,0,75,Riesgo,S,7,acebeda-la-id28001
1,2024-08-29 13:00:00,Nubes altas,24,24,29.0,0.0,0,57,Riesgo,0,75,Riesgo,SE,12,acebeda-la-id28001
2,2024-08-29 14:00:00,Muy nuboso,23,23,33.0,0.0,0,59,100,0,80,Riesgo,S,14,acebeda-la-id28001
3,2024-08-29 15:00:00,Nubes altas,23,23,35.0,0.0,0,55,Riesgo,0,80,Riesgo,S,17,acebeda-la-id28001
4,2024-08-29 16:00:00,Nubes altas,21,21,34.0,0.0,0,65,Riesgo,0,80,Riesgo,S,20,acebeda-la-id28001


In [7]:
#Limpiamos dfs
df_aemet[["localizacion","id_aemet"]] = df_aemet["localizacion_id"].str.rsplit("-",n=1,expand= True)
df_aemet.drop(columns = "localizacion_id",inplace= True)
df_municipios = df_municipios.T.rename(columns = {0:"Latitud",1:"Longitud",2:"Localidad"})
df_municipios.reset_index(inplace=True)

# Transformar "index", "municipio", "localización" a lista y hacer un diccionario para que todo sea igual

lista_municipios_aemet = list (df_aemet["localizacion"].unique())
lista_municipios_4sq = list(df_4sq["municipio"].unique())
lista_index_municipios = list(df_municipios["index"])

# Hacer un isin para ver que todos los municipios coinciden
#lista 4sq están todos en municipios
#{'cabrera-la', 'leganes', 'molinos-los'} no estan en municipios
set(lista_municipios_aemet).difference(set(lista_index_municipios))

# Filtrar las filas donde 'localizacion' es 'Cabrera-la', 'Leganes' o 'Molinos-los'
filtro = df_aemet.query('localizacion in ["cabrera-la", "leganes", "molinos-los"]')
df_aemet = df_aemet.drop(filtro.index)

#Examinamos valores extraños en df_aemet. La documentación oficial indica que el término "Riesgo" no es adecuado, pues se refiere a un término no deseado. Lo dropeamos
df_aemet["prob._precip.__(%)"].describe()
df_aemet["prob._precip.__(%)"].value_counts()

## Renombrar la columna actual
df_aemet = df_aemet.rename(columns={"prob._precip.__(%)": "Prob_precipitacion"})


##Drop Riesgo
filtro_riesgos = df_aemet.query('Prob_precipitacion in ["Riesgo"]')
df_aemet = df_aemet.drop(filtro_riesgos.index)

#Dropear avisos pues no aportan información (No sabemos de qué avisan)
df_aemet.drop(columns = ["avisos"],inplace = True)

#Relocalizamos IDAEMET al principio 
idaemet_col = df_aemet.pop('id_aemet')
df_aemet.insert(0, "id_aemet", idaemet_col)
df_aemet.columns
#df_aemet ok :)


df_municipios.reset_index(inplace=True)
df_municipios.rename(columns = {"level_0": "denominador"},inplace= True)
#df_municipios ok


#Nuevas columnas para establecer FKs: 
# Convertir las columnas 'index' y 'denominador' en un diccionario
diccionario_anadir = df_municipios.set_index('index')['denominador'].to_dict()

# Aplicar el mapeo en la columna 'localizacion' de df_aemet
df_aemet["denominado"] = df_aemet["localizacion"].map(diccionario_anadir)
df_4sq["denominado"] = df_4sq["municipio"].map(diccionario_anadir)


In [8]:
#Nos conectamos a postgres
try:
    conexion = psycopg2.connect(
        database = "Lab3M4",
        user = "postgres",
        password = "admin",
        host = "localhost",
        port = "5432"

)
except OperationalError:
    print("la contraseña es errónea")



In [ ]:
# Conexión a la base de datos
try:

    # Crear un cursor
    cursor = conexion.cursor()

    # Crear la tabla municipios sin UNIQUE en index
    query_table_municipios = """
    DROP TABLE IF EXISTS municipios CASCADE;

    CREATE TABLE IF NOT EXISTS municipios (
        denominador INT PRIMARY KEY,
        index VARCHAR(100),
        latitud INT,
        longitud INT,
        localidad VARCHAR(100)
    );
    """

    # Crear la tabla aemet referenciando el ID de municipios
    query_table_aemet = """
    DROP TABLE IF EXISTS aemet;

    CREATE TABLE IF NOT EXISTS aemet (
        id SERIAL PRIMARY KEY,
        id_aemet VARCHAR(50), 
        fecha TIMESTAMP,
        cielo VARCHAR(50),
        temp_c INT,
        sen_termica_c INT,
        racha_max_kmh DECIMAL,
        precipitacion_mm DECIMAL,
        nieve_mm INT,
        humedad_relativa INT,
        prob_precipitacion INT,
        prob_nieve INT,
        prob_tormenta INT,
        direccion_viento VARCHAR(50),
        velocidad_viento INT,
        denominado INT,
        localizacion VARCHAR (50),
            FOREIGN KEY (denominado) 
            REFERENCES municipios(denominador)
            ON UPDATE CASCADE
            ON DELETE CASCADE
    );
    """

    # Crear la tabla foursquare referenciando el ID de municipios
    query_table_4sq = """
    DROP TABLE IF EXISTS foursquare;

    CREATE TABLE IF NOT EXISTS foursquare (
        id SERIAL PRIMARY KEY,
        denominado INT,
        municipio VARCHAR(50),
        category VARCHAR(100),
        fsq_id VARCHAR(100),
        closed_bucket VARCHAR(50),
        distance DECIMAL,
        link TEXT,
        name VARCHAR(100),
        address TEXT,
        latitude DECIMAL,
        longitude DECIMAL,
        CONSTRAINT fk_municipio_foursquare
            FOREIGN KEY (denominado) 
            REFERENCES municipios(denominador)
            ON UPDATE CASCADE
            ON DELETE CASCADE
    );
    """

    # Crear las tablas
    cursor.execute(query_table_municipios)
    conexion.commit()

    cursor.execute(query_table_aemet)
    conexion.commit()

    cursor.execute(query_table_4sq)
    conexion.commit()

    # Cerrar el cursor y la conexión
    cursor.close()
    conexion.close()

except Exception as e:
    print(f"Ocurrió un error: {e}")
    if conexion:
        conexion.rollback()


In [10]:
def insertar_datos(df, insert_query):
    for row in df.itertuples(index=False, name=None):
        cursor.execute(insert_query, row)


In [11]:

# Intentar ejecutar el código y manejar excepciones
try:
    #Nos conectamos a postgres
    try:
        conexion = psycopg2.connect(
            database = "Lab3M4",
            user = "postgres",
            password = "admin",
            host = "localhost",
            port = "5432"

    )
    except OperationalError:
        print("la contraseña es errónea")

    # Crear un cursor
    cursor = conexion.cursor()

    # Insertar datos en la tabla "municipios"
    insert_municipios_query = """
        INSERT INTO municipios (denominador, index, latitud, longitud, localidad)
        VALUES (%s, %s, %s, %s, %s);
    """
    insertar_datos(df_municipios, insert_municipios_query)

    # Insertar datos en la tabla "aemet"
    insert_aemet_query = """
        INSERT INTO aemet (
            id_aemet,fecha, cielo, temp_c, sen_termica_c, racha_max_kmh, precipitacion_mm, nieve_mm, 
            humedad_relativa, prob_precipitacion, prob_nieve, prob_tormenta, direccion_viento, 
            velocidad_viento, localizacion, denominado
        ) VALUES (%s,%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
    """
    insertar_datos(df_aemet, insert_aemet_query)

    # Insertar datos en la tabla "foursquare"
    insert_foursquare_query = """
        INSERT INTO foursquare (
            municipio, category, fsq_id, closed_bucket, distance, link, name, address, latitude, longitude,denominado
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s,%s);
    """
    insertar_datos(df_4sq, insert_foursquare_query)

    # Confirmar los cambios
    conexion.commit()

    # Cerrar el cursor y la conexión
    cursor.close()
    conexion.close()

except Exception as e:
    print(f"Ocurrió un error: {e}")
    # Deshacer los cambios si ocurre un error
    if conexion:
        conexion.rollback()


# RECORDAMOS LAS PREGUNTAS:


-Eres una empresa de grabación que quiere grabar una película en un parque en los alreedores de Madrid, cuál es el sitio con el mejor tiempo en gral?

-En qué municipio hace más sol para grabar

-Dónde puedo encontrar un castillo para grabar un episodio medieval? Necesito que haya un estudio de películas cerca.

-Dónde hay más probabilidad de que nieve?

-Hay que evitar los lugares muy húmedos, pues afectan a las cámaras


In [23]:

#Nos conectamos a postgres
try:
        conexion = psycopg2.connect(
            database = "Lab3M4",
            user = "postgres",
            password = "admin",
            host = "localhost",
            port = "5432"

    )
except OperationalError:
        print("la contraseña es errónea")


#1: Cogemos el parque en el que el promedio de sensación térmica se acerque más a 20 grados:
cursor = conexion.cursor()
query1 = ("""
               SELECT 
    f.category,
    m.localidad, 
    m.latitud, 
    m.longitud, 
    AVG(a.sen_termica_c) AS promedio_sen_termica
FROM 
    aemet a
JOIN 
    municipios m ON a.denominado = m.denominador
JOIN
    foursquare f ON f.denominado = m.denominador
GROUP BY 
    m.localidad, m.latitud, m.longitud, f.category
HAVING
    f.category = 'Park'
ORDER BY 
    ABS(AVG(a.sen_termica_c) - 20)
LIMIT 1; 
               """)
cursor.execute(query1)

resultados = cursor.fetchall()
print(f"El parque con una temperatura agradable es {resultados}")

#2: Cogemos el municipio donde haya habido cielo despejado más horas
query2 = ("""
SELECT
    m.localidad,
    m.latitud,
    m.longitud,
    COUNT(a.cielo) AS cielos
FROM 
    aemet a
JOIN
    municipios m ON a.denominado = m.denominador
WHERE
    a.cielo = 'Cielo despejado'
GROUP BY
    m.localidad,
    m.latitud,
    m.longitud
order by
	cielos DESC
LIMIT 5;
""")

cursor.execute(query2)

resultados2 = cursor.fetchall()
print(f"El lugar con más cielos despejados es: {resultados2}")
#3 Escoger las localidades que tengan un castillo y un estudio de gabación
query3 =("""
SELECT 
    m.localidad, 
    m.latitud, 
    m.longitud
FROM
    municipios m
JOIN
    foursquare f ON f.denominado = m.denominador
WHERE
    f.category IN ('Castle', 'Film Studio')
GROUP BY
    m.localidad, m.latitud, m.longitud
         """)
cursor.execute(query3)

resultados3 = cursor.fetchall()
print(f"En {resultados3} hay catillos y estudios de grabación")

#4 Dónde ha nevado más?
query4 = ("""SELECT
    m.localidad,
    m.latitud,
    m.longitud,
    a.nieve_mm AS nevada
FROM 
    aemet a
JOIN
    municipios m ON a.denominado = m.denominador
GROUP BY
    m.localidad,
    m.latitud,
    m.longitud,
    a.nieve_mm 
order by
	nieve_mm asc 
LIMIT 5;
""")

cursor.execute(query4)

resultados4 = cursor.fetchall()
print(f"En {resultados4} ha nevado un cacho")

#5 Cuáles son los sitios más húmedos?
query5 = ("""
SELECT
    m.localidad,
    m.latitud,
    m.longitud,
    a.humedad_relativa AS humedad
FROM 
    aemet a
JOIN
    municipios m ON a.denominado = m.denominador
GROUP BY
    m.localidad,
    m.latitud,
    m.longitud,
    a.humedad_relativa 
order by
	humedad_relativa desc
LIMIT 5;
""")

resultado5 = cursor.fetchall()
print(f"En {resultado5} se pueden estropear las cámaras")

El parque con una temperatura agradable es [('Park', 'Meco', Decimal('40.553746'), Decimal('-3.3281561'), Decimal('19.9523809523809524'))]
El lugar con más cielos despejados es: [('Braojos', Decimal('41.0400124'), Decimal('-3.6425888'), 8), ('Puebla de la Sierra', Decimal('41.0121247'), Decimal('-3.4444367'), 8), ('Gascones', Decimal('41.0185654'), Decimal('-3.641499'), 7), ('Venturada', Decimal('40.7985257'), Decimal('-3.6207369'), 7), ('Patones', Decimal('40.8548194'), Decimal('-3.483954'), 6)]
En [('Batres', Decimal('40.2091393'), Decimal('-3.9215947')), ('Madrid', Decimal('40.4167047'), Decimal('-3.7035825')), ('Manzanares el Real', Decimal('40.7267632'), Decimal('-3.864981')), ('San Martín de Valdeiglesias', Decimal('40.3625'), Decimal('-4.3986503'))] hay catillos y estudios de grabación
En [('Daganzo de Arriba', Decimal('40.5451953'), Decimal('-3.45785'), 0), ('Parla', Decimal('40.2373952'), Decimal('-3.7718902'), 0), ('Campo Real', Decimal('40.3381881'), Decimal('-3.3813911'), 0

In [14]:
#Guardamos como csv para usarlos en Dbeaver
df_aemet.to_csv("../../Datos/lab3_df_aemet_bdd.csv", sep = ',', index = False, encoding= "utf-8")
df_municipios.to_csv("../../Datos/lab3_df_municipios_bdd.csv", sep = ',',index = False, encoding= "utf-8" )
df_4sq.to_csv("../../Datos/lab3_df_4sq_bdd.csv", sep = ',',index = False, encoding= "utf-8" )